This notebook is a backtest of the strategy outlined in the paper.

In [103]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import yaml
from datetime import date
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize
from sklearn.linear_model import LinearRegression

### Development

Do a per-stock fitting and signal generation, such that data pulling doesn't take too much time. If the fitted beta (market component) does not have data for the entire period of the given stock, then the fallback will be utilised.

In [3]:
load_dotenv()

# .env variables
db_url = os.getenv('DB_URL')
constituents_table = os.getenv('CONSTITUENTS_TABLE')
prices_table = os.getenv('PRICES_TABLE')

# connection to sql database
db_conn = create_engine(db_url)

# config yaml
with open('backtest_config.yaml') as f:
    backtest_cfg = yaml.load(f, Loader=yaml.FullLoader)

sector_betas = backtest_cfg['sector_betas']
backtest_settings = backtest_cfg['backtest']

Remove any duplicate constituents besides the most recent ones (since `yfinance` will only keep the most recent ones).

In [4]:
def fetch_patched_constituents():
    constituents = pd.read_sql(
        f"""
        SELECT * FROM {constituents_table}
        WHERE sector IS NOT NULL
        """,
        db_conn
    )

    available_constituents = pd.read_sql(
        f"""
        SELECT DISTINCT ticker FROM {prices_table}
        """,
        db_conn
    )["ticker"].to_list()

    constituents['end_date'] = pd.to_datetime(constituents['end_date'])
    constituents['end_date_filled'] = constituents['end_date'].fillna(pd.Timestamp.max)

    constituents = (
        constituents.sort_values(['ticker', 'end_date_filled'], ascending=[True, False])
        .drop_duplicates(subset='ticker', keep='first')
        .drop(columns='end_date_filled')
    )

    constituents = constituents[constituents['ticker'].isin(available_constituents)]

    return constituents.set_index('ticker').to_dict(orient='index')

constituents = fetch_patched_constituents()

In [231]:
available_constituents = pd.read_sql(
    f"""
    SELECT DISTINCT ticker FROM {prices_table}
    """,
    db_conn
)["ticker"].to_list()

In [232]:
constituents

{'A': {'start_date': datetime.date(2000, 6, 5),
  'end_date': NaT,
  'sector': 'Healthcare'},
 'AAL': {'start_date': datetime.date(2015, 3, 23),
  'end_date': Timestamp('2024-09-23 00:00:00'),
  'sector': 'Industrials'},
 'AAP': {'start_date': datetime.date(2015, 7, 9),
  'end_date': Timestamp('2023-08-25 00:00:00'),
  'sector': 'Consumer Discretionary'},
 'AAPL': {'start_date': datetime.date(1996, 1, 2),
  'end_date': NaT,
  'sector': 'Technology'},
 'ABBV': {'start_date': datetime.date(2013, 1, 2),
  'end_date': NaT,
  'sector': 'Healthcare'},
 'ABNB': {'start_date': datetime.date(2023, 9, 18),
  'end_date': NaT,
  'sector': 'Consumer Discretionary'},
 'ABT': {'start_date': datetime.date(1996, 1, 2),
  'end_date': NaT,
  'sector': 'Healthcare'},
 'ACGL': {'start_date': datetime.date(2022, 11, 1),
  'end_date': NaT,
  'sector': 'Financials'},
 'ACN': {'start_date': datetime.date(2011, 7, 6),
  'end_date': NaT,
  'sector': 'Technology'},
 'ADBE': {'start_date': datetime.date(1997, 5, 6

Now consider the generation of signals.

In [215]:
def generate_edge_leg_df(edge_leg: str) -> pd.DataFrame:
    """
    Generate signals for a given stock ticker (edge_leg).

    Args:
        edge_leg (str): The stock ticker for which signals are generated.

    Returns:
        pd.DataFrame: DataFrame containing sector ETF data for the given stock.
    """
    try:
        # Extract relevant config information for the given edge leg.
        edge_leg_start = constituents[edge_leg]['start_date']
        edge_leg_end = constituents[edge_leg]['end_date'] if constituents[edge_leg]['end_date'] is not pd.NaT else date.today()
        sector = constituents[edge_leg]['sector']
        sector_beta = sector_betas[sector]['beta']

        edge_data_query = f"""
            SELECT date, ticker, adj_close FROM adjusted_historical
            WHERE ticker = '{edge_leg}'
        """
        edge_data = pd.read_sql(edge_data_query, db_conn)

        sector_etf_query = f"""
            SELECT date, ticker, adj_close FROM adjusted_historical
            WHERE ticker = '{sector_beta}'
            AND date >= '{edge_leg_start}'
            AND date <= '{edge_leg_end}'
        """
        sector_etf_data = pd.read_sql(sector_etf_query, db_conn)

        # Ensure date column is in datetime format
        edge_data['date'] = pd.to_datetime(edge_data['date'])
        sector_etf_data['date'] = pd.to_datetime(sector_etf_data['date'])

        # We only want to test for days where the etf was also trading.
        edge_data = edge_data[edge_data['date'].isin(sector_etf_data['date'])]

        # Get (log) returns column for both edge and sector data
        edge_data['log_returns'] = np.log(edge_data['adj_close'] / edge_data['adj_close'].shift())
        sector_etf_data['log_returns'] = np.log(sector_etf_data['adj_close'] / sector_etf_data['adj_close'].shift())

        backtest_data = pd.concat([edge_data, sector_etf_data]).sort_values('date').dropna().reset_index(drop=True)

        return backtest_data
        

    except KeyError as e:
        print(f"KeyError: {e}. Check if edge_leg exists in constituents.")
        raise
    except Exception as e:
        print(f"An error occurred while generating signals: {e}")
        raise

def generate_signals(edge_leg_df: pd.DataFrame, edge_leg: str, hedge_leg: str) -> pd.DataFrame:
    """
    Returns the trading signals (as entry as well as s-score) for the edge leg.
    """
    def s_score(window_data: np.array) -> float:
        try:
            X_k = np.cumsum(window_data)                             

            X_n_plus_one = X_k[1:]                                     
            X_n = X_k[:-1]                                              

            ou_model = LinearRegression().fit(X_n.reshape(-1, 1), X_n_plus_one)

            # for clarity i define the parameters as found in the paper's residual estimation
            a = ou_model.intercept_
            b = ou_model.coef_               
            zeta = X_n_plus_one - ou_model.predict(X_n.reshape(-1, 1))

            m = a / (1 - b)
            kappa = -np.log(b) * 252
            if kappa < 252 / 30:
                raise Exception("Warning: estimated kappa values indicate undesired reversion time")

            if b < 0 or b > 0.9672:
                raise Exception("Warning: estimated b values in residual estimation process were anomalous")

            sigma_eq = np.sqrt(np.var(zeta) / (1 - b ** 2))


            return (-m / sigma_eq)[0]
        except Exception as e:
            return np.nan
    
    def rolling_apply(series: pd.Series, window: int, func) -> pd.Series:
        result = np.full(len(series), np.nan)
        arr = series.to_numpy()
        for i in range(window, len(arr)):
            result[i] = func(arr[i - window:i])
        return pd.Series(result, index=series.index)

    def fill_positions_from_s_score(s_score_series: pd.Series) -> pd.Series:
        """
        Implements Avellaneda & Lee threshold logic:
        - Entry short at >= +1.25, exit short at < +0.75
        - Entry long at <= -1.25, exit long at > -0.5
        """
        position = 0
        positions = []

        for score in s_score_series:
            if position == 0:
                if score >= 1.25:
                    position = -1  # Enter short
                elif score <= -1.25:
                    position = 1   # Enter long
            elif position == -1:
                if score < 0.75:
                    position = 0  # Exit short
            elif position == 1:
                if score > -0.5:
                    position = 0  # Exit long

            positions.append(position)

        return pd.Series(positions, index=s_score_series.index, name='position')

    W = backtest_settings['lookback']

    wide = (edge_leg_df
            .pivot(index='date', columns='ticker', values='log_returns')
            [[edge_leg, hedge_leg]]
            .sort_index()
            .dropna())
    y = wide[edge_leg]
    x = wide[hedge_leg]

    """
    Below is an efficient implementation for finding OLS residuals.
    It finds the rolling means, the rolling residual from the mean, and from this calculates
    the variance and covariance. This then allows us to find the beta and alpha.
    """
    mx = x.rolling(W, min_periods=W).mean()
    my = y.rolling(W, min_periods=W).mean()

    x_dev = x - mx
    y_dev = y - my

    var_x = (x_dev.pow(2)).rolling(W, min_periods=W).sum()
    cov_xy = (x_dev * y_dev).rolling(W, min_periods=W).sum()

    beta = (cov_xy / var_x).shift(1)
    alpha = (my - beta * mx).shift(1)

    fitted = alpha + beta * x
    residual = (y - fitted).dropna()

    """
    We then calculate the s-score. This is an estimation of the residual process and can be 
    found in the appendix of the paper.
    """

    s_score_results = rolling_apply(residual, W, s_score).ffill()
    s_score_results.name = 's_score'
    s_score_df = s_score_results.dropna().reset_index()

    s_score_df['edge_leg'] = edge_leg
    s_score_df['hedge_leg'] = hedge_leg
    s_score_df['signal'] = fill_positions_from_s_score(s_score_df['s_score'])

    # merge in log returns
    wide = wide.rename(columns={
        edge_leg: 'log_return_edge',
        hedge_leg: 'log_return_hedge'
    })

    return pd.merge(s_score_df, wide, on='date', how='left')

In [220]:
msft = generate_edge_leg_df('MSFT')
msft_pos = generate_signals(msft, 'MSFT', 'XLK')

In [225]:
msft_pos

,date,s_score,edge_leg,hedge_leg,signal,log_return_edge,log_return_hedge
0,1999-09-13,-3.115604,MSFT,XLK,1,-0.011916,-0.013723
1,1999-09-14,-2.376735,MSFT,XLK,1,0.012571,0.010849
2,1999-09-15,-1.409619,MSFT,XLK,1,-0.025975,-0.021084
3,1999-09-16,-0.475027,MSFT,XLK,0,0.014738,-0.004419
4,1999-09-17,-0.764158,MSFT,XLK,0,0.025600,0.019731
...,...,...,...,...,...,...,...
6496,2025-07-11,-0.542565,MSFT,XLK,1,0.003662,-0.004407
6497,2025-07-14,-0.994224,MSFT,XLK,1,-0.000596,-0.000821
6498,2025-07-15,-1.525619,MSFT,XLK,1,0.005551,0.008840
6499,2025-07-16,-2.325387,MSFT,XLK,1,-0.000395,0.003097


### Filling in all constituents

We now consider the performance of the strategy across ~25 years, across the entire universe available to us.

In [230]:
constituents.items()

dict_items([('A', {'start_date': datetime.date(2000, 6, 5), 'end_date': NaT, 'sector': 'Healthcare'}), ('AAL', {'start_date': datetime.date(2015, 3, 23), 'end_date': Timestamp('2024-09-23 00:00:00'), 'sector': 'Industrials'}), ('AAP', {'start_date': datetime.date(2015, 7, 9), 'end_date': Timestamp('2023-08-25 00:00:00'), 'sector': 'Consumer Discretionary'}), ('AAPL', {'start_date': datetime.date(1996, 1, 2), 'end_date': NaT, 'sector': 'Technology'}), ('ABBV', {'start_date': datetime.date(2013, 1, 2), 'end_date': NaT, 'sector': 'Healthcare'}), ('ABNB', {'start_date': datetime.date(2023, 9, 18), 'end_date': NaT, 'sector': 'Consumer Discretionary'}), ('ABT', {'start_date': datetime.date(1996, 1, 2), 'end_date': NaT, 'sector': 'Healthcare'}), ('ACGL', {'start_date': datetime.date(2022, 11, 1), 'end_date': NaT, 'sector': 'Financials'}), ('ACN', {'start_date': datetime.date(2011, 7, 6), 'end_date': NaT, 'sector': 'Technology'}), ('ADBE', {'start_date': datetime.date(1997, 5, 6), 'end_date': 

In [ ]:
positions = []

for ticker in available_constituents:
    print(ticker)
    try:
        edge_leg = ticker
        sector = constituents[ticker]['sector']
        hedge_leg = sector_betas[sector]['beta']

        edge_leg_df = generate_edge_leg_df(edge_leg)
        position_df = generate_signals(edge_leg_df, edge_leg, hedge_leg)
        positions.append(position_df)
    except Exception as e:
        print(e)

generate_signals() missing 2 required positional arguments: 'edge_leg' and 'hedge_leg'
generate_signals() missing 2 required positional arguments: 'edge_leg' and 'hedge_leg'
'BDK'
generate_signals() missing 2 required positional arguments: 'edge_leg' and 'hedge_leg'
generate_signals() missing 2 required positional arguments: 'edge_leg' and 'hedge_leg'
generate_signals() missing 2 required positional arguments: 'edge_leg' and 'hedge_leg'
generate_signals() missing 2 required positional arguments: 'edge_leg' and 'hedge_leg'
generate_signals() missing 2 required positional arguments: 'edge_leg' and 'hedge_leg'
'DDR'
generate_signals() missing 2 required positional arguments: 'edge_leg' and 'hedge_leg'
'COV'
generate_signals() missing 2 required positional arguments: 'edge_leg' and 'hedge_leg'
generate_signals() missing 2 required positional arguments: 'edge_leg' and 'hedge_leg'
generate_signals() missing 2 required positional arguments: 'edge_leg' and 'hedge_leg'
generate_signals() missin

KeyboardInterrupt: 

In [235]:
hedge_leg

'XLB'